In [ ]:
# ======================================================
# NOTEBOOK 6 — VGG16 Feature Extraction + Linear SVM
# NO PCA (simple version)
# ======================================================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as vgg_preprocess
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

print("TensorFlow:", tf.__version__)
print("GPU:", tf.config.list_physical_devices('GPU'))



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ------------------------------------------------------
# 1) Paths & Config
# ------------------------------------------------------
IMAGE_FOLDER = "/content/drive/MyDrive/HAM10000_images"   # where images live (Notebook1 should have created/cleaned this)
SPLITS_DIR = "/content/drive/MyDrive/splits"              # output from Notebook 1: df_train.csv, df_val.csv, df_test.csv
OUTPUT_DIR = "/content/drive/MyDrive/models"
os.makedirs(OUTPUT_DIR, exist_ok=True)

IMG_SIZE = 224
BATCH_SIZE = 32



In [ ]:
# ------------------------------------------------------
# 2) Load CSV splits
# ------------------------------------------------------
df_train = pd.read_csv(f"{SPLITS_DIR}/df_train.csv")
df_val   = pd.read_csv(f"{SPLITS_DIR}/df_val.csv")
df_test  = pd.read_csv(f"{SPLITS_DIR}/df_test.csv")

print(len(df_train), len(df_val), len(df_test))

# Ensure filepath exists
for df in [df_train, df_val, df_test]:
    if 'filepath' not in df.columns:
        df['filename'] = df['image_id'].astype(str) + '.jpg'
        df['filepath'] = df['filename'].apply(lambda x: os.path.join(IMAGE_FOLDER, x))




In [ ]:
# ------------------------------------------------------
# 3) Data generators (NO augmentation)
# ------------------------------------------------------
gen = ImageDataGenerator(preprocessing_function=vgg_preprocess)

flow_train = gen.flow_from_dataframe(
    df_train, x_col='filepath', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)

flow_val = gen.flow_from_dataframe(
    df_val, x_col='filepath', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)

flow_test = gen.flow_from_dataframe(
    df_test, x_col='filepath', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)

class_map = flow_train.class_indices
inv_class = {v:k for k,v in class_map.items()}
print("Classes:", inv_class)



In [ ]:
# ------------------------------------------------------
# 4) Load VGG16 feature extractor (GAP output = 512 dims)
# ------------------------------------------------------
base = VGG16(weights='imagenet', include_top=False,
             input_shape=(IMG_SIZE, IMG_SIZE, 3))

feat_model = Model(inputs=base.input,
                   outputs=GlobalAveragePooling2D()(base.output))

feat_model.summary()
3



In [ ]:
# ------------------------------------------------------
# 5) Feature extraction function
# ------------------------------------------------------
def extract_features(generator, model):
    steps = int(np.ceil(generator.n / generator.batch_size))
    features = model.predict(generator, steps=steps, verbose=1)
    labels = generator.classes
    return features, labels




In [ ]:
# ------------------------------------------------------
# 6) Extract features for train/val/test
# ------------------------------------------------------
X_train, y_train = extract_features(flow_train, feat_model)
X_val, y_val     = extract_features(flow_val, feat_model)
X_test, y_test   = extract_features(flow_test, feat_model)

print("Feature shape:", X_train.shape)   # expected (num_samples, 512)



In [ ]:
# ------------------------------------------------------
# 7) Build Linear SVM (simple, no PCA)
# ------------------------------------------------------
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

svm_model = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", LinearSVC(C=1, max_iter=5000))
])

svm_model.fit(X_train, y_train)


svm_model.fit(X_train, y_train)



In [ ]:
# ------------------------------------------------------
# 8) Validation Accuracy
# ------------------------------------------------------
y_val_pred = svm_model.predict(X_val)
acc_val = accuracy_score(y_val, y_val_pred)
print(f"\nValidation Accuracy: {acc_val*100:.2f}%")



In [ ]:
# ------------------------------------------------------
# 9) Test Accuracy + Report
# ------------------------------------------------------
y_test_pred = svm_model.predict(X_test)
acc_test = accuracy_score(y_test, y_test_pred)
print(f"\nTest Accuracy: {acc_test*100:.2f}%\n")

print("Classification Report:")
print(classification_report(y_test, y_test_pred, target_names=list(inv_class.values())))

# Confusion Matrix
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d",
            xticklabels=inv_class.values(),
            yticklabels=inv_class.values(),
            cmap="Blues")
plt.title("Confusion Matrix — VGG16 Features + SVM")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()



In [ ]:
# ------------------------------------------------------
# 10) Save SVM model
# ------------------------------------------------------
import joblib
joblib.dump(svm_model, f"{OUTPUT_DIR}/svm_vgg_features.joblib")

print("\nSaved model to:", f"{OUTPUT_DIR}/svm_vgg_features.joblib")



In [ ]:
# ------------------------------------------------------
# 11) Predict on uploaded image
# ------------------------------------------------------
from google.colab import files
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def predict_image(img_path):
    img = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    arr = img_to_array(img)
    arr = np.expand_dims(arr, axis=0)
    arr = vgg_preprocess(arr)

    feat = feat_model.predict(arr)
    pred = svm_model.predict(feat)[0]
    label = inv_class[pred]
    return label

print("Upload an image:")
uploaded = files.upload()

for fn in uploaded.keys():
    p = predict_image(fn)
    print(f"{fn} ➜ Predicted class: {p}")
